In [3]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from skimage import exposure
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from skimage.feature import hog
from skimage import exposure


In [9]:
# Hàm trích xuất đặc trưng HOG
def extract_hog_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fd, _ = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), visualize=True)  # Xóa `multichannel`
    return fd



# Cập nhật lại hàm extract_features
def extract_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    # Phát hiện đường thẳng
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=10)

    if lines is None:
        return [0, 0, 0]  # Không có đường nào phát hiện

    # Trích xuất các đặc trưng HOG
    hog_features = extract_hog_features(image)

    # Tính toán số lượng đường thẳng và khoảng cách
    vertical_lines = sorted([line[0][0] for line in lines if abs(line[0][0] - line[0][2]) < 5])
    horizontal_lines = sorted([line[0][1] for line in lines if abs(line[0][1] - line[0][3]) < 5])

    avg_v_distance = np.mean(np.diff(vertical_lines)) if len(vertical_lines) > 1 else 0
    avg_h_distance = np.mean(np.diff(horizontal_lines)) if len(horizontal_lines) > 1 else 0

    # Kiểm tra kích thước đặc trưng HOG và chuẩn hóa
    fixed_hog_size = 81  # Kích thước cố định mong muốn của đặc trưng HOG
    if len(hog_features) < fixed_hog_size:
        hog_features = np.pad(hog_features, (0, fixed_hog_size - len(hog_features)), mode='constant')
    else:
        hog_features = hog_features[:fixed_hog_size]  # Cắt nếu lớn hơn

    return [len(lines), avg_v_distance, avg_h_distance] + list(hog_features)


# Hàm đọc dữ liệu từ thư mục "board_images/"
def load_data():
    data = []
    labels = []
    
    # Kiểm tra xem thư mục có tồn tại không
    if not os.path.exists('board_images'):
        print("⚠️ Thư mục 'board_images' không tồn tại! Hãy chạy code tạo dữ liệu trước.")
        return [], [], None

    # Đọc ảnh từ thư mục
    file_list = os.listdir('board_images')
    if len(file_list) == 0:
        print("⚠️ Không có ảnh nào trong thư mục 'board_images'. Hãy kiểm tra dữ liệu!")
        return [], [], None

    for filename in file_list:
        img_path = os.path.join('board_images', filename)
        if img_path.endswith(".png"):
            # Đọc ảnh
            img = cv2.imread(img_path)
            
            # Kiểm tra ảnh có tồn tại không
            if img is None:
                print(f"⚠️ Lỗi khi đọc ảnh: {img_path}")
                continue

            # Trích xuất đặc trưng từ ảnh
            features = extract_features(img)

            # Gán nhãn từ tên file (ví dụ: "5x5_0.png" → label = "5x5")
            label = filename.split('_')[0]  

            # Lưu đặc trưng và nhãn
            data.append(features)
            labels.append(label)

    # Kiểm tra nếu không có dữ liệu
    if len(data) == 0:
        print("⚠️ Không có dữ liệu nào được tải vào. Kiểm tra lại thư mục ảnh!")
        return [], [], None

    # Chuyển đổi nhãn thành số
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)

    return np.array(data), np.array(labels), label_encoder



In [3]:
# Hàm huấn luyện mô hình
def train_model():
    # Đọc dữ liệu từ thư mục "board_images"
    data, labels, label_encoder = load_data()
    
    # Kiểm tra nếu dữ liệu trống
    if len(data) == 0 or len(labels) == 0:
        print("❌ Không thể huấn luyện mô hình vì không có dữ liệu!")
        return
    
    # Chia dữ liệu thành tập huấn luyện và kiểm tra (30% dữ liệu cho kiểm tra)
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
    
    # Huấn luyện mô hình SVM với kernel tuyến tính
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)
    
    # Dự đoán trên tập kiểm tra
    y_pred = model.predict(X_test)
    
    # Đánh giá mô hình bằng độ chính xác
    accuracy = accuracy_score(y_test, y_pred)
    print(f"✅ Độ chính xác của mô hình: {accuracy * 100:.2f}%")

    # Hiển thị kết quả dự đoán so với thực tế
    print("\n🔹 Kết quả dự đoán (tối đa 10 kết quả):")
    for i in range(min(len(y_test), 10)):  # Hiển thị tối đa 10 kết quả
        print(f"Thực tế: {label_encoder.inverse_transform([y_test[i]])[0]}, Dự đoán: {label_encoder.inverse_transform([y_pred[i]])[0]}")

    # Trả về mô hình đã huấn luyện và label encoder
    return model, label_encoder

# Chạy mô hình huấn luyện
train_model()

✅ Độ chính xác của mô hình: 83.33%

🔹 Kết quả dự đoán (tối đa 10 kết quả):
Thực tế: 11x11, Dự đoán: 13x13
Thực tế: 8x8, Dự đoán: 8x8
Thực tế: 14x14, Dự đoán: 14x14
Thực tế: 3x3, Dự đoán: 3x3
Thực tế: 10x10, Dự đoán: 10x10
Thực tế: 6x6, Dự đoán: 6x6
Thực tế: 15x15, Dự đoán: 15x15
Thực tế: 12x12, Dự đoán: 11x11
Thực tế: 11x11, Dự đoán: 11x11
Thực tế: 11x11, Dự đoán: 11x11


(SVC(kernel='linear'), LabelEncoder())

In [10]:
def train_model_svm():
    # Đọc dữ liệu
    data, labels, label_encoder = load_data()

    if len(data) == 0 or len(labels) == 0:
        print("❌ Không thể huấn luyện mô hình vì không có dữ liệu!")
        return

    # Chia dữ liệu thành tập huấn luyện và kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
    
    # Tìm kiếm tham số tối ưu cho SVM
    parameters = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10], 'gamma': [0.1, 1, 'scale']}
    grid_search = GridSearchCV(SVC(), parameters, cv=5)
    
    grid_search.fit(X_train, y_train)
    
    # Sử dụng mô hình tốt nhất
    best_model = grid_search.best_estimator_

    # Dự đoán trên tập kiểm tra
    y_pred = best_model.predict(X_test)
    
    # Đánh giá mô hình
    accuracy = accuracy_score(y_test, y_pred)
    print(f"✅ Độ chính xác của mô hình SVM: {accuracy * 100:.2f}%")

    # Hiển thị kết quả dự đoán
    print("\n🔹 Kết quả dự đoán:")
    for i in range(min(len(y_test), 10)):  # Hiển thị tối đa 10 kết quả
        print(f"Thực tế: {label_encoder.inverse_transform([y_test[i]])[0]}, Dự đoán: {label_encoder.inverse_transform([y_pred[i]])[0]}")

train_model_svm()

✅ Độ chính xác của mô hình SVM: 92.22%

🔹 Kết quả dự đoán:
Thực tế: 11x11, Dự đoán: 11x11
Thực tế: 8x8, Dự đoán: 8x8
Thực tế: 14x14, Dự đoán: 14x14
Thực tế: 3x3, Dự đoán: 3x3
Thực tế: 10x10, Dự đoán: 10x10
Thực tế: 6x6, Dự đoán: 6x6
Thực tế: 15x15, Dự đoán: 15x15
Thực tế: 12x12, Dự đoán: 12x12
Thực tế: 11x11, Dự đoán: 11x11
Thực tế: 11x11, Dự đoán: 11x11


In [ ]:
import time

start_time = time.time()

grid_search.fit(X_train, y_train)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"⏱️ Thời gian huấn luyện: {elapsed_time:.2f} giây")

print(f"Số mẫu trong tập kiểm tra: {len(X_test)}")

import time

# Đo thời gian xử lý
start_time = time.time()

# Dự đoán trên tập kiểm tra
y_pred = best_model.predict(X_test)

end_time = time.time()
elapsed_time = end_time - start_time

# Tính thời gian trung bình xử lý mỗi mẫu
avg_time_per_sample = elapsed_time / len(X_test)

print(f"⏱️ Thời gian xử lý cho {len(X_test)} mẫu: {elapsed_time:.2f} giây")
print(f"⏱️ Thời gian trung bình xử lý mỗi mẫu: {avg_time_per_sample:.4f} giây")

for i in range(min(len(y_test), 10)):
    print(f"Thực tế: {label_encoder.inverse_transform([y_test[i]])[0]}, Dự đoán: {label_encoder.inverse_transform([y_pred[i]])[0]}")


In [8]:
import joblib

def train_model_svm():
    # Đọc dữ liệu
    data, labels, label_encoder = load_data()

    if len(data) == 0 or len(labels) == 0:
        print("❌ Không thể huấn luyện mô hình vì không có dữ liệu!")
        return

    # Chia dữ liệu thành tập huấn luyện và kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
    
    # Tìm kiếm tham số tối ưu cho SVM
    parameters = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10], 'gamma': [0.1, 1, 'scale']}
    grid_search = GridSearchCV(SVC(), parameters, cv=5)
    
    grid_search.fit(X_train, y_train)
    
    # Sử dụng mô hình tốt nhất
    best_model = grid_search.best_estimator_

    # Lưu mô hình ra file
    joblib.dump(best_model, "svm_caro_model.pkl")
    print("✅ Mô hình đã được lưu thành công vào 'svm_caro_model.pkl'")

    # Dự đoán trên tập kiểm tra
    y_pred = best_model.predict(X_test)
    
    # Đánh giá mô hình
    accuracy = accuracy_score(y_test, y_pred)
    print(f"✅ Độ chính xác của mô hình SVM: {accuracy * 100:.2f}%")

train_model_svm()


⚠️ Thư mục 'board_images' không tồn tại! Hãy chạy code tạo dữ liệu trước.
❌ Không thể huấn luyện mô hình vì không có dữ liệu!


In [ ]:
def train_model_rf():
    # Đọc dữ liệu
    data, labels, label_encoder = load_data()
   
    if len(data) == 0 or len(labels) == 0:
        print("❌ Không thể huấn luyện mô hình vì không có dữ liệu!")
        return

    # Chia dữ liệu thành tập huấn luyện và kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
    
    # Huấn luyện mô hình Random Forest
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    
    # Sử dụng mô hình tốt nhất
    best_model = rf_model.base_estimator_

     # Lưu mô hình ra file
    joblib.dump(best_model, "svm_caro_model.pkl")
    print("✅ Mô hình đã được lưu thành công vào 'svm_caro_model.pkl'")

    # Dự đoán trên tập kiểm tra
    y_pred = rf_model.predict(X_test)
    
    # Đánh giá mô hình
    accuracy = accuracy_score(y_test, y_pred)
    print(f"✅ Độ chính xác của mô hình Random Forest: {accuracy * 100:.2f}%")

    # Hiển thị kết quả dự đoán
    print("\n🔹 Kết quả dự đoán:")
    for i in range(min(len(y_test), 10)):  # Hiển thị tối đa 10 kết quả
        print(f"Thực tế: {label_encoder.inverse_transform([y_test[i]])[0]}, Dự đoán: {label_encoder.inverse_transform([y_pred[i]])[0]}")


train_model_rf()

AttributeError: 'RandomForestClassifier' object has no attribute 'base_estimator_'

In [6]:
def train_model_dt():
    # Đọc dữ liệu
    data, labels, label_encoder = load_data()

    if len(data) == 0 or len(labels) == 0:
        print("❌ Không thể huấn luyện mô hình vì không có dữ liệu!")
        return

    # Chia dữ liệu thành tập huấn luyện và kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
    
    # Huấn luyện mô hình Decision Tree
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train)
    
    # Dự đoán trên tập kiểm tra
    y_pred = dt_model.predict(X_test)
    
    # Đánh giá mô hình
    accuracy = accuracy_score(y_test, y_pred)
    print(f"✅ Độ chính xác của mô hình Decision Tree: {accuracy * 100:.2f}%")

    # Hiển thị kết quả dự đoán
    print("\n🔹 Kết quả dự đoán:")
    for i in range(min(len(y_test), 10)):  # Hiển thị tối đa 10 kết quả
        print(f"Thực tế: {label_encoder.inverse_transform([y_test[i]])[0]}, Dự đoán: {label_encoder.inverse_transform([y_pred[i]])[0]}")

train_model_dt()

✅ Độ chính xác của mô hình Decision Tree: 91.11%

🔹 Kết quả dự đoán:
Thực tế: 11x11, Dự đoán: 11x11
Thực tế: 8x8, Dự đoán: 8x8
Thực tế: 14x14, Dự đoán: 14x14
Thực tế: 3x3, Dự đoán: 3x3
Thực tế: 10x10, Dự đoán: 10x10
Thực tế: 6x6, Dự đoán: 6x6
Thực tế: 15x15, Dự đoán: 15x15
Thực tế: 12x12, Dự đoán: 12x12
Thực tế: 11x11, Dự đoán: 11x11
Thực tế: 11x11, Dự đoán: 11x11


In [ ]:
import os
from collections import Counter

# Kiểm tra số lượng ảnh trong thư mục
def check_data_distribution():
    label_counts = Counter()
    
    for filename in os.listdir('board_images'):
        if filename.endswith('.png'):
            label = filename.split('_')[0]  # Lấy nhãn từ tên file
            label_counts[label] += 1

    print("📊 Phân bố dữ liệu huấn luyện:")
    for label, count in label_counts.items():
        print(f"  - {label}: {count} ảnh")

# Chạy kiểm tra dữ liệu
check_data_distribution()


In [ ]:
import cv2
import numpy as np
import os
from skimage.feature import hog
from skimage import exposure
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Hàm trích xuất đặc trưng HOG
def extract_hog_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Chuyển ảnh sang grayscale
    fd, _ = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), visualize=True)  # XÓA `channel_axis=-1`
    return fd



# Hàm trích xuất đặc trưng từ ảnh
def extract_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    # Phát hiện đường thẳng
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=10)

    if lines is None:
        return [0, 0, 0]  # Không có đường nào phát hiện

    # Trích xuất các đặc trưng HOG
    hog_features = extract_hog_features(image)

    # Tính toán số lượng đường thẳng và khoảng cách
    vertical_lines = sorted([line[0][0] for line in lines if abs(line[0][0] - line[0][2]) < 5])
    horizontal_lines = sorted([line[0][1] for line in lines if abs(line[0][1] - line[0][3]) < 5])

    avg_v_distance = np.mean(np.diff(vertical_lines)) if len(vertical_lines) > 1 else 0
    avg_h_distance = np.mean(np.diff(horizontal_lines)) if len(horizontal_lines) > 1 else 0

    return [len(lines), avg_v_distance, avg_h_distance] + list(hog_features)

# Hàm đọc dữ liệu từ thư mục "board_images/"
def load_data():
    data, labels = [], []
    
    # Kiểm tra xem thư mục có tồn tại không
    if not os.path.exists('board_images/'):
        print("⚠️ Thư mục 'board_images' không tồn tại! Hãy kiểm tra lại đường dẫn.")
        return [], [], None

    # Đọc ảnh từ thư mục
    file_list = os.listdir('board_images/')
    if len(file_list) == 0:
        print("⚠️ Không có ảnh nào trong thư mục 'board_images'. Hãy kiểm tra dữ liệu!")
        return [], [], None

    for filename in file_list:
        img_path = os.path.join('board_images', filename)
        if img_path.endswith(".png"):
            img = cv2.imread(img_path)

            if img is None:
                print(f"⚠️ Lỗi khi đọc ảnh: {img_path}")
                continue

            # Trích xuất đặc trưng
            features = extract_features(img)
            label = filename.split('_')[0]  # Ví dụ: "5x5_0.png" → label = "5x5"

            data.append(features)
            labels.append(label)

    if len(data) == 0:
        print("⚠️ Không có dữ liệu nào được tải vào. Kiểm tra lại thư mục ảnh!")
        return [], [], None

    # Chuyển đổi nhãn thành số
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)

    return np.array(data), np.array(labels), label_encoder

# Hàm huấn luyện chung
def train_model(model_name="svm_default"):
    # Đọc dữ liệu
    data, labels, label_encoder = load_data()

    if len(data) == 0 or len(labels) == 0:
        print("❌ Không thể huấn luyện mô hình vì không có dữ liệu!")
        return

    # Chia dữ liệu thành tập huấn luyện và kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

    # Chọn mô hình
    if model_name == "svm_default":
        model = SVC(kernel='linear')
    elif model_name == "svm_tuned":
        parameters = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10], 'gamma': [0.1, 1, 'scale']}
        grid_search = GridSearchCV(SVC(), parameters, cv=5)
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
    elif model_name == "random_forest":
        model = RandomForestClassifier(n_estimators=100, random_state=42)
    elif model_name == "decision_tree":
        model = DecisionTreeClassifier(random_state=42)
    else:
        print("❌ Mô hình không hợp lệ! Vui lòng chọn 'svm_default', 'svm_tuned', 'random_forest', hoặc 'decision_tree'.")
        return

    # Huấn luyện mô hình
    model.fit(X_train, y_train)

    # Dự đoán trên tập kiểm tra
    y_pred = model.predict(X_test)

    # Đánh giá mô hình
    accuracy = accuracy_score(y_test, y_pred)
    print(f"✅ Độ chính xác của mô hình {model_name}: {accuracy * 100:.2f}%")

    # Hiển thị kết quả dự đoán
    print("\n🔹 Kết quả dự đoán:")
    for i in range(min(len(y_test), 10)):  # Hiển thị tối đa 10 kết quả
        print(f"Thực tế: {label_encoder.inverse_transform([y_test[i]])[0]}, Dự đoán: {label_encoder.inverse_transform([y_pred[i]])[0]}")

    return model

# Huấn luyện từng mô hình
if __name__ == "__main__":
    print("\n🔷 Huấn luyện mô hình SVM mặc định...")
    train_model("svm_default")

    print("\n🔷 Huấn luyện mô hình SVM (tuning tham số)...")
    train_model("svm_tuned")

    print("\n🔷 Huấn luyện mô hình Random Forest...")
    train_model("random_forest")

    print("\n🔷 Huấn luyện mô hình Decision Tree...")
    train_model("decision_tree")                                                                 



🔷 Huấn luyện mô hình SVM mặc định...


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (900,) + inhomogeneous part.